# Dentro de Airbnb, explorando conjunto de datos


**Explorando Buenos Aires a través de los Datos de Airbnb (Julio-Agosto- Septiembre 2023)**

Buenos Aires, Argentina, es una ciudad que nunca deja de sorprender. En el invierno de 2023, miles de viajeros exploraron sus calles, disfrutaron de su cultura y se sumergieron en experiencias únicas. Descubriremos esta hermosa ciudad a través de los datos de Airbnb recopilados durante el trimestre de julio a septiembre de 2023.
En este informe, abordaremos el proceso de análisis y responderemos algunas preguntas claves.

Antes de comenzar cualquier análisis, es fundamental identificar las fuentes de datos disponibles.

ref: http://insideairbnb.com/get-the-data/

El siguiente conjunto de datos que exploraremos contiene datos sobre Airbnb en Buenos Aires, Argentina, (Período: Julio - Agosto - Septiembre 2023).
Información resumida y métricas para Buenos Aires.

`argentina.csv`

Trataremos de obtener algunas respuestas:
- Total de alojamientos realizados
- Alojamientos por tipo de habitación
- Cantidad de propietarios
- Promedio del precio
- Promedio del precio por tipo de alojamiento
- Promedio de ingresos
- Cantidad y % de alojamientos relizados por barrio
- Cantidad de alojamientos que tuvo cada anfitrión
- % de propietarios con un alojamiento
- Anfitriones Top
- Mapa interactivo de los alojamientos realizados en BsAs
- Algunas conclusiones...
- Visualización final

## 1- Descarga de los datos

Librerías

In [ ]:
# importar los módulos para el análisis de datos: pandas y numpy
import numpy as np  # para cálculos en Python
import pandas as pd # para estructura de datos
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

In [ ]:
## seleccionar la ruta del dataset "df_argentina.csv"
path='.../argentina.csv'
df_argentina = pd.read_csv(path)
print('Datos descargados y leídos con éxito!')

In [ ]:
# osbservamos los primeros cinco elementos
df_argentina.head()

In [ ]:
# los últimos cinco elementos
df_argentina.tail()

In [ ]:
# analizamos la dimensión del dataframe
print(df_argentina.shape)

In [ ]:
# breve resumen del dataframe
df_argentina.info(verbose=False)

In [ ]:
# información general del dataframe
print('Cantidad de Filas y columnas:', df_argentina.shape, '\n')
print('Nombre de las columnas:',df_argentina.columns, '\n')
print('Información sobre el dataset:', '\n')
df_argentina.info()

In [ ]:
# índices utilizados
df_argentina.index

## 2- Limpieza de los datos

In [ ]:
# buscar duplicados
df_argentina.duplicated().value_counts

In [ ]:
# identificar valores faltantes
df_argentina.isnull().values.any()

Dado que el resultado es True podemos confirmar que existen valores faltantes.
Visualicemos a continación cuales son las columnas con valores faltantes.

In [ ]:
print(df_argentina.isnull().sum())

Según el resultado, podemos afirmar que:

- La variable **"neighbourhood_group"** contiene **29,346 registros vacíos**.
- La variable **"last_review"** contiene **5,505 registros vacíos**.
- La variable **"reviews_per_month"** contiene **5,505 registros vacíos**.
- La variable **"license"** contiene **28,901 registros vacíos**.




Veamos los valores únicos para cada columna.
El resultado proporciona información valiosa sobre la diversidad de valores en cada columna del conjunto de datos.

In [ ]:
# valores únicos para cada columna
df_argentina.nunique()

Contemos cuántas columnas existen de tipo numérico y cuántas de tipo objeto. En total tiene que haber 18 columnas.

In [ ]:
# contar tipos de datos
numericos = df_argentina.select_dtypes(include=['int', 'float']).shape[1]
objetos = df_argentina.select_dtypes(include='object').shape[1]
print(f"Número de columnas numéricas: {numericos}")
print(f"Número de columnas de tipo objeto: {objetos}")

Eliminaremos a continuación las columnas que no necesitamos para nuestro análisis

In [ ]:
# eliminamos las columnas
df_argentina.drop(['neighbourhood_group', 'last_review', 'reviews_per_month', 'number_of_reviews_ltm', 'license'], axis=1, inplace=True)

# visualizamos nuevamente el dataframe para ver los cambios
df_argentina.head()

In [ ]:
# visualizamos nuevamente la dimensión del dataframe, para chequear que tenga 5 columnas menos (de 18 a 13)
print('dimensión de los datos:', df_argentina.shape)

Cambiemos el nombre de las columnas para un mejor entendimiento de los datos

In [ ]:
# cambiar nombre de las columnas
df_argentina.rename(columns={'name':'nombre', 'host_id':'id_anfitrion', 'host_name':'nombre_anfitrion', 'neighbourhood':'barrio', 'room_type':'tipo_habitacion', 'price':'precio', 'minimum_nights':'minimo_noches', 'number_of_reviews':'numero_reseñas', 'availability_365':'disponibilidad'}, inplace=True)
df_argentina.head()

Visualicemos los tipos de datos de nuestro df para verificar.

In [ ]:
# tipos de datos por columna
print(df_argentina.dtypes)

Chequear que todas las etiquetas de las columnas sean de tipo string, para asegurar la coherencia

In [ ]:
# chequeamos los tipos de etiqueta
all(isinstance(column, str) for column in df_argentina.columns)

El resultado True es una confirmación de que los nombres de las columnas en el DataFrame son cadenas de texto, lo cual es el tipo de dato esperado para las etiquetas de las columnas en la mayoría de los DataFrames de pandas. Esto es importante para garantizar que las operaciones y análisis posteriores del DataFrame se realicen de manera adecuada y consistente.

In [ ]:
print("\nResumen estadístico:")
print(df_argentina.describe())

El resumen estadístico proporciona información valiosa sobre las características de las columnas numéricas del conjunto de datos.

Algunas observaciones:

- La columna "precio" tiene un valor medio (media) de aproximadamente 24,780.22 y una desviación estándar significativamente alta, lo que indica una gran variabilidad en los precios de los alojamientos.

- La columna "minimo_noches" tiene un valor mínimo de 1 y un valor máximo de 1,000, lo que sugiere una variabilidad significativa en los requisitos de estadías mínimas.

- La columna "numero_reseñas" tiene un valor medio de alrededor de 22.12 y una desviación estándar considerable, lo que indica una variabilidad en la cantidad de reseñas.

- La columna "disponibilidad" tiene un valor mínimo de 0 y un valor máximo de 365, lo que sugiere que existen alojamientos con disponibilidad durante todo el año.

In [ ]:
df_filtrado = df_argentina[df_argentina['minimo_noches'] >=999]
df_filtrado

## 3- Exploración de los datos y visualizaciones

Comencemos a responder algunas preguntas:

### ↪ Total de alojamientos realizados

Cantidad total de alojamientos realizados en el período (trimestre: julio-agosto-setiembre 2023)

In [ ]:
# Calcular la cantidad total de alojamientos que se han realizado
total_alojamientos = df_argentina.shape[0]

# Mostrar la cantidad total de alojamientos
print(f"La cantidad total de alojamientos es: {total_alojamientos}")

### ↪ Alojamientos por tipo de habitación

Los anfitriones de Airbnb pueden publicar casas/apartamentos completos, habitaciones privadas, compartidas y más...

En el análisis de datos de Airbnb, una parte fundamental es entender la distribución de los diferentes tipos de habitación que los anfitriones ofrecen a los huéspedes. Este análisis arroja luz sobre las preferencias de alojamiento y puede ser útil tanto para los anfitriones que desean tomar decisiones informadas sobre sus listados como para los viajeros que buscan la opción que mejor se adapte a sus necesidades. El conjunto de datos que estamos examinando proporciona información detallada sobre la cantidad y el porcentaje de cada tipo de habitación ofrecida.

Analicemos los tipos de alojamientos del dataframe.

In [ ]:
# Calcular la cantidad por tipo de habitación
cant_tipo_habitacion = df_argentina['tipo_habitacion'].value_counts()

# Calcular el porcentaje por tipo de habitación
porc_tipo_habitacion = (cant_tipo_habitacion / len(df_argentina)) * 100

# Crear un nuevo DataFrame con los resultados
df_resultado = pd.DataFrame({
    'Tipo de Habitación': cant_tipo_habitacion.index,
    'Cantidad': cant_tipo_habitacion.values,
    'Porcentaje (%)': porc_tipo_habitacion.values
})

# Ordenar el DataFrame por cantidad en orden descendente
df_resultado = df_resultado.sort_values(by='Cantidad', ascending=False)

# Mostrar el resultado
print(df_resultado)


Creamos un gráfico para visualizar mejor los resultados

In [ ]:
# Crear un gráfico de barras interactivo
fig = px.bar(df_resultado, x='Tipo de Habitación', y='Cantidad', text='Porcentaje (%)',
             title='Cantidad y Porcentaje por Tipo de Habitación (trimestre julio - agosto - setiembre 2023)')

# Personalizar el diseño del gráfico
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# Mostrar el gráfico interactivo
fig.show()

**Resumen de la Distribución de Tipos de Habitación en Airbnb**

El conjunto de datos revela que hay cuatro tipos principales de habitaciones ofrecidas en Airbnb, cada uno con su propia participación en el mercado:

- **Hogar o Apartamento Completo (Entire home/apt):**
   - Cantidad: 26,467
   - Porcentaje: 90.19%

- **Habitación Privada (Private room):**
   - Cantidad: 2,565
   - Porcentaje: 8.74%

- **Habitación Compartida (Shared room):**
   - Cantidad: 215
   - Porcentaje: 0.73%

- **Habitación de Hotel (Hotel room):**
   - Cantidad: 99
   - Porcentaje: 0.34%

**Interpretación y Conclusiones:**

- **Hogar o Apartamento Completo:** El tipo "Entire home/apt" es claramente el tipo de habitación más popular, representando el 90.19% de todas las ofertas. Esto indica que la mayoría de los anfitriones optan por alquilar sus viviendas completas, lo que sugiere que los viajeros pueden estar buscando privacidad y espacio para sí mismos.

- **Preferencia por Habitaciones Privadas:** Las "Private rooms" también tienen una presencia significativa, con un 8.74% de las ofertas. Esto indica que hay una demanda considerable de habitaciones privadas dentro de los hogares de los anfitriones, posiblemente para viajeros que buscan un equilibrio entre privacidad y presupuesto.

- **Habitaciones Compartidas y de Hotel:** Las "Shared rooms" y las "Hotel rooms" representan una proporción relativamente pequeña del mercado, con 0.73% y 0.34%, respectivamente. Esto sugiere que estos tipos de alojamiento pueden no ser tan populares entre los viajeros, ya que las preferencias se inclinan hacia opciones más privadas.


### ↪ Cantidad de propietarios

In [ ]:
# Calcular la cantidad de propietarios únicos reflejados en el set de datos
cantidad_propietarios = df_argentina['id_anfitrion'].nunique()

print("Cantidad de propietarios:", cantidad_propietarios)

El resultado muestra que en el conjunto de datos, hay un total de 14958 propietarios únicos. Esto significa que hay 14958 personas o entidades diferentes que han listado propiedades en Airbnb en este caso en la ciudad de Buenos Aires en el trimestre.

### ↪ Calculamos el promedio del precio

In [ ]:
# Calcular el promedio de la variable precio
promedio_precio = df_argentina['precio'].mean()

print("Promedio de precio:", promedio_precio)

El resultado muestra que el promedio de precio en el conjunto de datos para Bs. As. es de aproximadamente 24,780.22. Este valor representa un promedio de los precios de todas las propiedades listadas en el conjunto de datos.

El cálculo del promedio es una métrica importante en el análisis de datos, ya que proporciona una comprensión general de la tendencia de los precios en el mercado. En este caso, el promedio de precio podría ser útil para entender cuál es el precio promedio que los viajeros pueden esperar pagar por una propiedad en Argentina según los datos del conjunto. Esto es útil tanto para los anfitriones que establecen sus tarifas como para los viajeros que buscan alojamiento en la región.

### ↪ Promedio del precio por tipo de alojamiento

In [ ]:
# Calcular el promedio de precio por tipo de alojamiento
promedio_por_tipo = df_argentina.groupby('tipo_habitacion')['precio'].mean()

print("Promedio de precio por tipo de alojamiento:")
print(promedio_por_tipo)

El resultado muestra el promedio de precio para cada tipo de alojamiento en el conjunto de datos:

- "Entire home/apt" (Casa o apartamento completo): $24,781.75

- "Hotel room" (Habitación de hotel): $24,737.43

- "Private room" (Habitación privada): $21,604.18

- "Shared room" (Habitación compartida): $62,502.69

Este análisis es esencial para comprender cómo varían los precios según el tipo de alojamiento en Bs. As. Ayuda tanto a los anfitriones a establecer tarifas competitivas como a los viajeros a elegir la opción que mejor se adapte a su presupuesto y preferencias.

La gran diferencia en el precio promedio entre "Shared room" y los otros tipos de alojamiento podría indicar una mayor demanda o una oferta limitada en esa categoría.

Graficamos para visualizar mejor

In [ ]:
# Colores personalizados
colores_barras = ['skyblue', 'lightgreen', 'lightcoral', 'lightblue', 'pink']

# Crear un gráfico de barras
plt.figure(figsize=(10, 6))  # Tamaño del gráfico
ax = promedio_por_tipo.plot(kind='bar', color=colores_barras)

# Crear un estilo de fondo personalizado
plt.style.use({
    'axes.facecolor': 'whitesmoke',  # Color de fondo del gráfico
    'axes.edgecolor': 'lightgrey',  # Color del borde del gráfico
    'grid.color': 'white',  # Color de las líneas de cuadrícula
    'axes.labelcolor': 'black'  # Color de las etiquetas
    })

# Personalizar el gráfico
plt.title('Promedio de Precio por Tipo de Alojamiento Airbnb en Buenos Aires', y=1.06)
plt.xlabel('Tipo de Alojamiento', labelpad=15)
plt.ylabel('Precio Promedio', labelpad=15)
plt.xticks(rotation=0)  # Rotar etiquetas en el eje X para mayor legibilidad

# Alinear y ajustar los espaciados internos
plt.tight_layout()

# Agregar números en las barras con un mayor espaciado vertical y horizontal
for i, v in enumerate(promedio_por_tipo):
    ax.text(i, v, f'{v:.0f}', ha='center', va='bottom', fontsize=9, color='black', rotation='horizontal')

# Mostrar el gráfico
plt.show()


### ↪ Promedio de ingresos

Calculamos el promedio de ingreso del set de datos (recordar que el df contiene la información de un trimestre: julio-agosto-setiembre 2023).

Se tiene como objetivo calcular el ingreso promedio generado por los alojamientos en el conjunto de datos df_argentina.

In [ ]:
# Calcular el ingreso total
df_argentina['ingreso_total'] = df_argentina['precio'] * df_argentina['minimo_noches']
ingreso_total = df_argentina['ingreso_total'].sum()

# Calcular el número total de alojamientos
total_alojamientos = len(df_argentina)

# Calcular el ingreso promedio
ingreso_promedio = ingreso_total / total_alojamientos

print("Ingreso promedio:", ingreso_promedio)

El resultado muestra que el ingreso promedio por alojamiento en el conjunto de datos de Airbnb para Bs.As. Argentina es de aproximadamente 188,728.35. Este valor representa el promedio de los ingresos generados por cada alojamiento en la ciudad.

### ↪ Cantidad y % de alojamientos realizados por barrio

In [ ]:
# Agrupar el DataFrame por el campo 'barrio' y contar la cantidad de alojamientos en cada barrio.
alojamientos_por_barrio = df_argentina['barrio'].value_counts().reset_index()
alojamientos_por_barrio.columns = ['barrio', 'cantidad_alojamientos']

# Calcular el porcentaje de alojamientos en cada barrio.
total_alojamientos = len(df_argentina)
alojamientos_por_barrio['porcentaje'] = (alojamientos_por_barrio['cantidad_alojamientos'] / total_alojamientos) * 100

# Ordenar el DataFrame por 'cantidad_alojamientos' en orden descendente.
alojamientos_por_barrio = alojamientos_por_barrio.sort_values(by='cantidad_alojamientos', ascending=False)

# Imprimir el resultado.
print(alojamientos_por_barrio)

El resultado proporciona una visión detallada de la distribución de alojamientos en diferentes barrios de Bs.As. Argentina. Cada fila del DataFrame muestra el nombre del barrio, la cantidad de alojamientos en ese barrio y el porcentaje de alojamientos que representa con respecto al total.

Este análisis es esencial para comprender qué áreas o barrios son más populares o tienen una mayor concentración de alojamientos en Airbnb en la ciudad de Bs. As. en el trimestre. Puede ser útil tanto para viajeros que buscan alojamiento en áreas específicas como para anfitriones que desean comprender la competencia en su área.

Elaboramos un gráfico para visualizar mejor el resultado

In [ ]:
# Establecer un estilo de gráfico de barras
sns.set(style="whitegrid")

# Crear un gráfico de barras horizontal para una mejor legibilidad
plt.figure(figsize=(12, 8))
ax = sns.barplot(x="cantidad_alojamientos", y="barrio", data=alojamientos_por_barrio, palette="viridis")

# Agregar etiquetas y título
plt.xlabel("Cantidad de Alojamientos realizados")
plt.ylabel("Barrio")
plt.title("Cantidad de Alojamientos realizados por Barrio trimestre julio-agosto-setiembre 2023")

# Mostrar los valores en las barras
for p in ax.patches:
    width = p.get_width()
    plt.text(width, p.get_y() + p.get_height() / 2, f"{int(width)}", ha="left", va="center")

plt.tight_layout()
plt.show()


### ↪ Cantidad de alojamientos que tuvo cada anfitrion y barrio

In [ ]:
# Agrupar el DataFrame por 'id_anfitrion' y contar la cantidad de alojamientos.
alojamientos_por_anfitrion = df_argentina.groupby(['id_anfitrion', 'nombre_anfitrion'])['id'].count().reset_index()
alojamientos_por_anfitrion.columns = ['id_anfitrion', 'nombre_anfitrion', 'cantidad_alojamientos']

# Encontrar la ubicación de cada anfitrión.
ubicacion_por_anfitrion = df_argentina.groupby(['id_anfitrion', 'nombre_anfitrion'])['barrio'].first().reset_index()

# Combinar los DataFrames en uno solo.
resultado = pd.merge(alojamientos_por_anfitrion, ubicacion_por_anfitrion, on=['id_anfitrion', 'nombre_anfitrion'])

# Crear un widget de DataFrame interactivo
df_widget = widgets.Output()
with df_widget:
    display(resultado)

# Mostrar el widget
df_widget


Esta información es valiosa para comprender la actividad de los anfitriones y su distribución geográfica. Los usuarios pueden navegar por la tabla para explorar detalles sobre cada anfitrión y sus alojamientos.

Exportemos el resultado a un archivo de excel

In [ ]:
# from google.colab import files

## Nombre del archivo de Excel de salida
# archivo_excel = "resultado_anfitriones.xlsx"

# Exportar los datos a un archivo Excel
# resultado.to_excel(archivo_excel, index=False)

## Descargar el archivo automáticamente
# files.download(archivo_excel)


Apliquemos el filtro por "id_anfitrión" para poder ver la cantidad de alojamientos por anfitrión y barrios

In [ ]:
# Agrupar el DataFrame por 'id_anfitrion' y contar la cantidad de alojamientos.
alojamientos_por_anfitrion = df_argentina.groupby(['id_anfitrion', 'nombre_anfitrion'])['id'].count().reset_index()
alojamientos_por_anfitrion.columns = ['id_anfitrion', 'nombre_anfitrion', 'cantidad_alojamientos']

# Encontrar la ubicación de cada anfitrión.
ubicacion_por_anfitrion = df_argentina.groupby(['id_anfitrion', 'nombre_anfitrion'])['barrio'].first().reset_index()

# Combinar los DataFrames en uno solo.
resultado = pd.merge(alojamientos_por_anfitrion, ubicacion_por_anfitrion, on=['id_anfitrion', 'nombre_anfitrion'])

# Crear un widget de filtro por id_anfitrion
filtro_id_anfitrion = widgets.Dropdown(
    options=['Todos'] + resultado['id_anfitrion'].unique().tolist(),
    value='Todos',
    description='ID del Anfitrión:'
)

# Función para aplicar el filtro
def filtrar_datos(id_anfitrion):
    if id_anfitrion == 'Todos':
        datos_filtrados = resultado
    else:
        datos_filtrados = resultado[resultado['id_anfitrion'] == id_anfitrion]

    with output:
        output.clear_output()
        display(datos_filtrados)

# Aplicar el filtro al interactuar con el widget
output = widgets.Output()
display(filtro_id_anfitrion, output)
filtro_id_anfitrion.observe(lambda change: filtrar_datos(change.new), names='value')


### ↪ % de propietarios con un alojamiento

In [ ]:
# Identificar propietarios únicos
prop_unicos = df_argentina[['id_anfitrion', 'nombre_anfitrion']].drop_duplicates()

# Contar la cantidad de propietarios únicos
total_prop_unicos = prop_unicos.shape[0]

# Identificar propietarios únicos con exactamente 1 alojamiento
prop_un_alojamiento = df_argentina.groupby(['id_anfitrion', 'nombre_anfitrion'])['id'].count().reset_index()
prop_un_alojamiento = prop_un_alojamiento[prop_un_alojamiento['id'] == 1]

# Contar la cantidad de propietarios con exactamente 1 alojamiento
total_prop_un_alojamiento = prop_un_alojamiento.shape[0]

# Calcular el porcentaje de propietarios con 1 alojamiento
porc_prop_un_alojamiento = (total_prop_un_alojamiento / total_prop_unicos) * 100

print(f"Porcentaje de propietarios con 1 alojamiento: {porc_prop_un_alojamiento:.2f}%")

El resultado muestra que aproximadamente el 76.77% de los propietarios en el conjunto de datos de Airbnb en Bs.As. en el trimestre tienen exactamente 1 alojamiento listado. Esto significa que la gran mayoría de los propietarios tienen una sola propiedad para alquilar.

Este análisis puede ser útil para comprender la diversidad de propietarios en la plataforma y cuántos de ellos tienen un enfoque en un solo alojamiento.

### ↪ Anfitriones Top

Lista de anfitriones Top basada en el dataframe de Bs.As. trimestre julio-agosto-setiembre 2023

In [ ]:
# Calcular la cantidad de alojamientos por anfitrión y ordenar en orden descendente
anfitriones_top = df_argentina.groupby(['id_anfitrion', 'nombre_anfitrion'])['id'].count().reset_index()
anfitriones_top.columns = ['id_anfitrion', 'nombre_anfitrion', 'cantidad_alojamientos']
anfitriones_top = anfitriones_top.sort_values(by='cantidad_alojamientos', ascending=False)

# Seleccionar los 20 anfitriones principales (ajusta el número según tus necesidades)
top_anfitriones = anfitriones_top.head(20)

# Mostrar la lista de los 20 anfitriones principales
print("Los 20 anfitriones con la mayor cantidad de alojamientos:")
print(top_anfitriones)

### ↪ Qué nos pueden decir las reseñas?

Analicemos un poco el tema de las reseñas

In [ ]:
# visualicemos nuevamente el df_argentina para chequear su estado
df_argentina.head()

Calcular la mediana y el promedio de reseñas:

In [ ]:
# Calcular la mediana y el promedio de reseñas
mediana_resenas = df_argentina['numero_reseñas'].median()
promedio_resenas = df_argentina['numero_reseñas'].mean()

print(f"Mediana de reseñas: {mediana_resenas}")
print(f"Promedio de reseñas: {promedio_resenas}")

En base a los resutlados podemos sacar algunas observaciones:

- La mediana es significativamente más baja que el promedio, lo que sugiere que hay algunos alojamientos con un gran número de reseñas que están inflando el promedio.

- El promedio más alto indica que algunos alojamientos han recibido un gran número de reseñas, posiblemente debido a su popularidad o tiempo en la plataforma. Al mismo tiempo, la mediana más baja indica que la mayoría de los alojamientos tienen un número relativamente bajo de reseñas.

- Los alojamientos extremadamente populares con un gran número de reseñas pueden estar sesgando el promedio hacia arriba. Esto significa que unos pocos alojamientos con muchas reseñas pueden estar teniendo un impacto significativo en el promedio.

- La diferencia entre la mediana y el promedio sugiere que hay una variación significativa en la calidad y popularidad de los alojamientos en el conjunto de datos. Algunos alojamientos son muy populares y tienen muchas reseñas, mientras que otros tienen menos reseñas.

Explorar la distribución de reseñas  utilizando un gráfico para visualizar cómo se distribuyen las reseñas en el conjunto de datos y determinar si hay una concentración en un rango particular de valores.

In [ ]:
# Establecer el estilo de la gráfica con Seaborn
sns.set_style("whitegrid")

# Crear un histograma
plt.figure(figsize=(10, 6))  # Ajustar el tamaño de la figura
plt.hist(df_argentina['numero_reseñas'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
plt.xlabel('Número de Reseñas')
plt.ylabel('Frecuencia')
plt.title('Distribución de Reseñas en Bs As')

# Agregar una línea vertical para la mediana
median = df_argentina['numero_reseñas'].median()
plt.axvline(median, color='red', linestyle='dashed', linewidth=2, label=f'Mediana: {median}')

# Agregar una leyenda
plt.legend()

# Mostrar un cuadro con estadísticas
plt.text(150, 1000, f'Mediana: {median}', bbox=dict(facecolor='white', alpha=0.8))

# Personalizar el diseño
plt.grid(True, linestyle='--', alpha=0.7)
sns.despine(left=True)  # Quitar los bordes izquierdos para una apariencia más limpia

# Mostrar el histograma
plt.show()

Estudiemos un poco las correlaciones

In [ ]:
# Calcula la matriz de correlación de Pearson (variables numéricas)
correlac_matrix = df_argentina.corr()

# Filtra las correlaciones de 'numero_reseñas' con otras variables
correlac_resenas = correlac_matrix['numero_reseñas']

# Imprime las correlaciones
print("Correlaciones con 'numero_reseñas':")
print(correlac_resenas)

EL resultado deja evidencia de que no hay correlaciones fuertes entre la variable "numero_reseñas" y el resto.
Por ejemplo una correlación de -0.010 está muy cerca de cero, lo que sugiere que el precio y el número de reseñas no tienen una relación lineal significativa. Esto significa que, en general, el precio de un alojamiento no parece estar fuertemente relacionado con la cantidad de reseñas que recibe. En otras palabras, un precio más alto o más bajo no parece influir significativamente en la cantidad de reseñas que los alojamientos reciben.

Alojamientos con pocas o ninguna reseña:

In [ ]:
# Filtrar alojamientos con menos de 5 reseñas ordenados de mayor a menor
aloj_pocas_resenas = df_argentina[df_argentina['numero_reseñas'] < 5].sort_values(by='numero_reseñas', ascending=False)

# Imprimir los primeros 10 alojamientos con pocas reseñas
print(aloj_pocas_resenas.head(10))

Alojamientos populares:

In [ ]:
# Filtrar alojamientos con más de 100 reseñas (ajusta el valor según tu criterio), orden descendente
aloj_populares = df_argentina[df_argentina['numero_reseñas'] > 100].sort_values(by='numero_reseñas', ascending=False)

# Mostrar los primeros 10 alojamientos populares en una tabla
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.max_colwidth', None)  # Mostrar todo el contenido de las celdas

print("Primeros 10 alojamientos populares (ordenados por número de reseñas en orden descendente):")
display(aloj_populares.head(10))


Comparaciones por tipo de alojamiento o ubicación:

In [ ]:
# Comparar el número de reseñas entre diferentes tipos de alojamiento
resenas_por_tipo_ordenado = df_argentina.groupby('tipo_habitacion')['numero_reseñas'].mean().sort_values(ascending=False)

print("Número medio de reseñas por tipo de habitación (ordenado en orden descendente):")
print(resenas_por_tipo_ordenado)

Dado los resutlados el tipo de habitación con mayor número de reseñas es el tipo de habitación "Entire home/apt", con un promedio de aproximadamente 22.92 reseñas. Esto sugiere que, en promedio, alojamientos de este tipo reciben más reseñas que los demás.
Comprender cómo los diferentes tipos de habitación impactan en la cantidad de reseñas que reciben. Por ejemplo, es común que los alojamientos completos o "Entire home/apt" reciban más reseñas que las habitaciones compartidas o "Shared room". Esto puede deberse a las experiencias que los huéspedes tienen en diferentes tipos de alojamientos.

In [ ]:
# Comparar el número de reseñas entre diferentes ubicaciones
resenas_por_ubicacion = df_argentina.groupby('barrio')['numero_reseñas'].mean().sort_values(ascending=False)

print(resenas_por_ubicacion)

Algunas observaciones basadas en el resultado:

Los barrios "San Nicolas", "Palermo" y "San Telmo" tienen los números medios más altos de reseñas, con valores de aproximadamente 28.98, 27.44 y 25.35, respectivamente. Esto sugiere que los alojamientos en estos barrios tienden a recibir más reseñas en promedio.

Hay una amplia variabilidad en el número medio de reseñas entre los diferentes barrios. Mientras que algunos barrios como "San Nicolas" y "Palermo" tienen números medios significativamente altos, otros como "Villa Real", "Villa Riachuelo" y "Villa Soldati" tienen números medios de reseñas muy bajos o incluso cero.

Esta información puede ser útil para comprender la popularidad relativa de los barrios en términos de alojamientos con reseñas. Los barrios más populares pueden ser atractivos para los turistas y viajeros, lo que podría explicar el mayor número de reseñas.

Es importante tener en cuenta que la cantidad de reseñas puede depender de varios factores, como la ubicación, la oferta de alojamientos, la afluencia de turistas y más. Los resultados proporcionan una visión general de la actividad de reseñas por barrio en el conjunto de datos, pero se deben considerar otros factores para obtener una comprensión completa.




Recordar:
La interpretación de los resultados debe realizarse en función del contexto de los datos y los objetivos de análisis específicos.
Tener en cuenta que estos resultados reflejan una relación observada en los datos y no implican necesariamente causalidad.

Anfitriones destacados:

In [ ]:
# Agrupar el DataFrame por el nombre del anfitrión y calcular la cantidad de alojamientos y el número total de reseñas para cada anfitrión.
anfitriones_destacados = df_argentina.groupby('nombre_anfitrion').agg({'id': 'count', 'numero_reseñas': 'sum'}).reset_index()

# Renombrar las columnas para mayor claridad.
anfitriones_destacados.columns = ['nombre_anfitrion', 'cantidad_alojamientos', 'cantidad_reseñas']

# Ordenar los anfitriones destacados por el número de alojamientos en orden descendente.
anfitriones_destacados = anfitriones_destacados.sort_values(by='cantidad_alojamientos', ascending=False)

# Mostrar los 10 anfitriones destacados (puedes ajustar el número según tus necesidades).
print(anfitriones_destacados.head(10))

### ↪ Mapa interactivo Buenos Aires | Airbnb

Mapa interactivo de Buenos Aires con todos los marcadores de los alojamientos realizados.

Utilizaremos la librería folium para crear el mapa interactivo.
Los alojamientos están agrupados en clústeres para facilitar la visualización de múltiples puntos en el mapa. Los usuarios pueden hacer clic en los marcadores para ver más detalles sobre cada alojamiento. Este tipo de visualización es útil para identificar la distribución geográfica de los alojamientos en la ciudad y puede contener información valiosa para los usuarios que buscan alojamiento en Buenos Aires.

In [ ]:
#!pip3 install folium==0.5.0
import folium

print('Folium instalado e importado!')

In [ ]:
from folium import plugins

# variables para latitud y longitud de la ciudad de Buenos Aires (Argentina)
latitud_bsas = -34.61315
longitud_bsas = -58.37723

# crear mapa
mapa_bsas = folium.Map(location = [latitud_bsas, longitud_bsas], zoom_start = 12)

# crear una instancia de un objeto de grupo de marcas para los alojamientos en el df
alojamientos = plugins.MarkerCluster().add_to(mapa_bsas)

# recorrer el marco de datos y agreguar cada punto de datos al grupo de marcas
for lat, lng, label, in zip(df_argentina.latitude, df_argentina.longitude, df_argentina.nombre):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(alojamientos)

# visualizar el mapa
mapa_bsas

### 🔊 Algunas conclusiones...

🔹 `Destinos`:

Buenos Aires ofrece una variedad de barrios únicos para explorar. Los datos revelan cuáles fueron los destinos más buscados durante este período:

- San Nicolás: este barrio céntrico con casi 29 reseñas de media, fue uno de los destinos favoritos de los viajeros.
- Palermo: Con una media de 27.44 reseñas, atrae a aquellos que buscan experiencias de moda y estilo de vida.
- San Telmo: Con una media de 25.35 reseñas, te sumerge en la historia y la cultura de la ciudad.

🔹 `Variedad de Alojamiento:`

Los viajeros tuvieron una amplia gama de opciones de alojamiento en Buenos Aires. Los datos revelan cuáles fueron los tipos de alojamiento más populares:

- Alojamiento completo (Entire home/apt): Con una media de 22.92 reseñas, esta opción ofrece privacidad y comodidad.
- Habitación privada (Private room): Con una media de 15.58 reseñas, estas habitaciones ofrecen una experiencia más íntima.
- Habitación de hotel (Hotel room): Con 14.59 reseñas de media, los hoteles ofrecen comodidades adicionales y servicio.


🔹 `Anfitriones Destacados:`

Encontramos a algunos anfitriones excepcionales que se destacaron por su hospitalidad y cantidad de alojamientos:

- Anfitrión Destacado 1: **Pablo** - Con 420 alojamientos y 7914 reseñas.
- Anfitrión Destacado 2: **Federico** - Con 409 alojamientos y 7294 reseñas.
- Anfitrión Destacado 3: **Diego** - Con 350 alojamientos y 8290 reseñas.

### ↪ Visualización final

In [ ]:
# Descargamos el df_argentina para utilizarlo en Power BI, para nuestro Dashboard

from google.colab import files

# Nombre del archivo de Excel de salida
datosBsAs = "datosBsAs.xlsx"

# Exportar los datos a un archivo Excel
df_argentina.to_excel(datosBsAs, index=False)

# Descargar el archivo automáticamente
files.download(datosBsAs)

Visualización en proceso...